In [3]:
from basic_tools import *

In [7]:
command='gcta64 --bfile {} --make-grm --out {} --thread-num 40'.format(final_plink_aa_path,final_plink_aa_path+'.grm')
command

'gcta64 --bfile data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged --make-grm --out data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged.grm --thread-num 40'

In [9]:
command='gcta64 --HEreg --grm {} --pheno data/out_assoc/height/phenotype.phe --out data/out_assoc/height/HE --thread-num 40'.format(final_plink_aa_path+'.grm')
command

'gcta64 --HEreg --grm data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged.grm --pheno data/out_assoc/height/phenotype.phe --out data/out_assoc/height/HE --thread-num 40'

In [ ]:
awk '{print $3}' data/out_assoc/height/phenotype.phe > temp.phe;
paste data/out_assoc/height/phenotype.phe temp.phe > temp2.phe
head temp2.phe

In [ ]:
gcta64 --HEreg-bivar 1 2 --grm data/genotype/4_merge/grm --pheno temp2.phe --out data/out_assoc/height/HE2 --thread-num 40